In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm.notebook.tqdm as tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

## Se importan los dataframes 

In [ ]:
# load all metadata
#hau nire diskan daukat SAD karpetan 2020-2021

video_games = []#
insurances = []   
#dfMergedfMeta= pd.read_csv('Cell_Phones/SamsungAppleXiaomiReviews_prueba.csv')
video_games= pd.read_csv('VG-Reviews5AndMetaElecNintSonyMic_v2.csv')
insurances = pd.read_csv('HRBlockIntuitReviewsTrainDev_vLast7.csv', low_memory=False)


In [ ]:
video_games.keys()

In [ ]:
video_games['brand'].value_counts()

In [ ]:
insurances.keys()

In [ ]:
insurances['brand'].value_counts()

## Se obtienen en dataframes aparte las distintas marcas

In [ ]:
dfNintendo = video_games[video_games['brand'].str.contains('Nintendo',na=False)]

In [ ]:
dfSony = video_games[video_games['brand'].str.contains('Sony|PlayStation|Electr',na=False)]

In [ ]:
dfMicrosoft = video_games[video_games['brand'].str.contains('[M|m]icrosoft',na=False)]

In [ ]:
dfIntuit = insurances[insurances['brand'].str.contains('Intuit')]

In [ ]:
dfHR = insurances[~insurances['brand'].str.contains('Intuit')]

## Se separan las reviews negativas y positivas de cada dataframe

In [ ]:
dfNintendoPositives=dfNintendo[dfNintendo['overall']>3]
dfNintendoNegatives=dfNintendo[dfNintendo['overall']<=3]

In [ ]:
dfSonyPositives=dfSony[dfSony['overall']>3]
dfSonyNegatives=dfSony[dfSony['overall']<=3]

In [ ]:
dfMicrosoftPositives=dfMicrosoft[dfMicrosoft['overall']>3]
dfMicrosoftNegatives=dfMicrosoft[dfMicrosoft['overall']<=3]

In [ ]:
dfIntuitPositives = dfIntuit[dfIntuit['overall']>3]
dfIntuitNegatives = dfIntuit[dfIntuit['overall']<=3]

In [ ]:
dfHRPositives = dfHR[dfHR['overall']>3]
dfHRNegatives = dfHR[dfHR['overall']<=3]

## Se obtienen las text reviews

In [ ]:
nintendoPositivesDocuments = dfNintendoPositives[dfNintendoPositives['reviewText'].notna()]['reviewText'].tolist()
nintendoNegativesDocuments = dfNintendoNegatives[dfNintendoNegatives['reviewText'].notna()]['reviewText'].tolist()

In [ ]:
sonyPositivesDocuments = dfSonyPositives[dfSonyPositives['reviewText'].notna()]['reviewText'].tolist()
sonyNegativesDocuments = dfSonyNegatives[dfSonyNegatives['reviewText'].notna()]['reviewText'].tolist()

In [ ]:
microsoftPositivesDocuments = dfMicrosoftPositives[dfMicrosoftPositives['reviewText'].notna()]['reviewText'].tolist()
microsoftNegativesDocuments = dfMicrosoftNegatives[dfMicrosoftNegatives['reviewText'].notna()]['reviewText'].tolist()

In [ ]:
intuitPositivesDocuments = dfIntuitPositives[dfIntuitPositives['reviewText'].notna()]['reviewText'].tolist()
intuitNegativesDocuments = dfIntuitNegatives[dfIntuitNegatives['reviewText'].notna()]['reviewText'].tolist()

In [ ]:
HRPositivesDocuments = dfHRPositives[dfHRPositives['reviewText'].notna()]['reviewText'].tolist()
HRNegativesDocuments = dfHRNegatives[dfHRNegatives['reviewText'].notna()]['reviewText'].tolist()

## LDA

In [ ]:
def display_topics(H, W, feature_names, documents, test, no_top_words = 30, no_top_documents = 10):
    f = open(test + ".txt", "a")
    for topic_idx, topic in enumerate(H):
        
        topic = "Topic %d:" % (topic_idx)
        f.write(topic + '\n\n')
        print(topic)
        
        feature_names = " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
        f.write(feature_names + '\n\n')
        print(feature_names)
        
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])
            f.write(documents[doc_index] + '\n')
    f.close()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### LDAHRPos_11_0.8

In [ ]:
no_topics = 11

alpha = 0.8
beta = 0.8

documents = HRPositivesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDAHRPos_11_0.8')

### LDAHRNeg_6_0.8

In [ ]:
no_topics = 6

alpha = 0.8
beta = 0.8

documents = HRNegativesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDAHRNeg_6_0.8')

### LDAIntuitPos_16_0.8

In [ ]:
no_topics = 16

alpha = 0.8
beta = 0.8

documents = intuitPositivesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDAIntuitPos_16_0.8')

### GensimMicroPos

### GensimMicroNeg